# Pivot database table to one row per timestamp per room instead of several rows for each sensor.

In [ ]:
import sqlite3

with sqlite3.connect('room_data.db') as conn:
    cursor = conn.cursor()

    try:
        conn.execute("BEGIN TRANSACTION")

        cursor.execute("""
        CREATE TABLE IF NOT EXISTS sensor_data_history (
            room_id INTEGER,
            timestamp TEXT,
            temperature REAL,
            airquality REAL,
            daylight REAL,
            light REAL,
            PRIMARY KEY (room_id, timestamp)
        )
        """)

        cursor.execute("""
        INSERT OR REPLACE INTO sensor_data_history (room_id, timestamp, temperature, airquality, daylight, light)
        SELECT 
            room_id,
            strftime('%Y-%m-%d %H:%M:%S', timestamp) AS timestamp,
            MAX(CASE WHEN sensor = 'temperature' THEN value END) AS temperature,
            MAX(CASE WHEN sensor = 'airquality' THEN value END) AS airquality,
            MAX(CASE WHEN sensor = 'daylight' THEN value END) AS daylight,
            MAX(CASE WHEN sensor = 'light' THEN value END) AS light
        FROM sensor_data
        GROUP BY room_id, timestamp
        """)

        cursor.execute("""
        DELETE FROM sensor_data
        WHERE (room_id, timestamp) IN (
            SELECT room_id, timestamp
            FROM sensor_data_history
        )
        """)

        conn.commit()

    except sqlite3.Error as e:
        conn.rollback()
        print(f"Database error: {e}")
        raise